In [10]:
import Bio.SeqIO
import requests

In [14]:
url = "http://flybase.org/cgi-bin/getseq.html?source=dmel&id=FBgn0004666&chr=3R&dump=PrecompiledFasta&targetset=exon"
r = requests.get( url )
# r.response
# ?r
buf = r.text.splitlines()

In [284]:
url = "http://flybase.org/cgi-bin/getseq.html?source=dmel&id=FBgn0004666&chr=3R&dump=PrecompiledFasta&targetset=exon"
import Bio.SeqIO
import urllib
# with urllib.urlopen(url) as response:
response = urllib.urlopen(url)
fas = [x for x in Bio.SeqIO.parse(response,'fasta')]
f_ind = lambda seq: int( seq.id.split(':')[-1])
fas0 = sorted(fas, key= f_ind)
#     html = response.read()


In [32]:
# f_start = lambda seq: seq.loc.split(':')[-1].split('..')
f_ind = lambda seq: int( seq.id.split(':')[-1])
fas0 = sorted(fas, key= f_ind)
# fas[0]

In [77]:
# regex.findall()
import re
s = str(fas[0].seq)
# help(s)
# res  = re.search( re.compile("TAG"), s)#
res  = re.match( "TAG", s)

In [102]:
import re
p =  re.compile("TAG")

In [85]:
p.match(s).groupdict()

{}

In [86]:
s

'TAGTCACTCATTCGCTCGCCGCCCGCTCGCTGGTTCAGCTGCGCGTCCACGCCGTCTGTCCTTATATACATATATGTATATTACCGATATATATAATATCCATACACATATATCCCAGTTTGGATAATTTTCTCGCTGTGCCTACAAGTGTTAAATAATATATATAAATATATAAAAATATCTGTGTGCATTAGAAATAAGTCAAGAATTCTAGCAAGCAAATTGCCCAAGTGCCATAAACGCAATCGCATAATACCCGTTAACGGGCTCATTAGGAAAACCTTTTCCGGCAAAAACACAAAACTCGCTCGGCCGCAACACACATTTAAGTTTATTTATTTTAAAACCATCGACTTAAAGCAAAAATGTGCTAGCAGTTAAGGCGACAATTTAGTTTGATTCTCTCGGTGAACGGAGCTTTCCTATTGGGATTTATCGACGGTGGCCCAACTGCAAGAATACCAGCGGCAAAACGGATAGCTGCCAGAG'

In [73]:
# help(res)
# res.lastindex()
res.groupdict()
# res

{}

In [34]:
########## Report all possible variants from the  TSS  ########

######################################################
###  Find all stop codons in ###############
######################################################
######################################################
fas

[SeqRecord(seq=Seq('TAGTCACTCATTCGCTCGCCGCCCGCTCGCTGGTTCAGCTGCGCGTCCACGCCG...GAG', SingleLetterAlphabet()), id='FBgn0004666:1', name='FBgn0004666:1', description='FBgn0004666:1 type=exon; loc=3R:13057755..13058243; parent=FBgn0004666,FBtr0082711,FBtr0334614; MD5=11e7e7dfbf81129056e38c394728cf5e; release=r6.17; species=Dmel; length=489;', dbxrefs=[]),
 SeqRecord(seq=Seq('AGTGTTGGATCAAGTCGCACTGCACCCAGTGACCAAGTCGTCGTCGTCGTTGTA...AAA', SingleLetterAlphabet()), id='FBgn0004666:2', name='FBgn0004666:2', description='FBgn0004666:2 type=exon; loc=3R:13062522..13062730; parent=FBgn0004666,FBtr0334613; MD5=3c91cf3f661220346ba19626c7d877fa; release=r6.17; species=Dmel; length=209;', dbxrefs=[]),
 SeqRecord(seq=Seq('GGCTCAGTTGCAAACAGGTGATTGCAGGGATATGAGCAAGTGCTGAGAAGGTGC...ATG', SingleLetterAlphabet()), id='FBgn0004666:3', name='FBgn0004666:3', description='FBgn0004666:3 type=exon; loc=3R:13072399..13072550; parent=FBgn0004666,FBtr0082710; MD5=ab0b8578bd622434d153738549192fac; release=r6.17; specie

In [104]:
# p = re.compile("ATG")
# p.finditer(s)

<callable-iterator at 0x7f95e5ca92d0>

In [287]:
# res = []
import re
import itertools

kf = lambda x:x[1]
def find_posframe( pt, s):
# pt = pstop
    d = {0: -1,1: -1,2: -1}
    d = {0: [-1],1: [-1],2: [-1]}
    pos = [ m.start() for m in pt.finditer( s)]
    frame = [x % 3 for x in pos]
    it = itertools.groupby(sorted(zip(pos,frame),key = kf),kf)
    d.update({k: [obj[0] for obj in gp] for k,gp in it})
#     d.update({k: min(obj[0] for obj in gp) for k,gp in it})
    return [d[0],d[1],d[2]]

def find_start(s):
    d = {0: -1,1: -1,2: -1}
    pos = [ m.start() for m in pt.finditer( s)]
    frame = [x % 3 for x in pos]
    

# print d
#         print next(gp)
#     sorted(pos,frame)
#     itertools.groupby()
#     for 
#     return zip(pos,frame)
# print find_posframe("TAA|TAG|TGA",s).__len__()




pstart = re.compile("ATG")
pstop  = re.compile("TAA|TAG|TGA")
# print find_posframe(pstop,s)
# class segment( Bio.SeqRecord.SeqRecord):
class segment(object):
    def __init__(self, seqRec, head = 0):
        self.seqRec = seqRec
        self.starts =  find_posframe( pstart, str( self.seqRec.seq ))
        self.stops =  find_posframe(  pstop,   str( self.seqRec.seq ))
        self.shift = len(self.seqRec) % 3
#         if head:
#     def as_head(self):
        if head:
#             for k,v in self.stops.items():
            for k,v in enumerate(self.stops):
                mv = max(v)
                lst = [x for x in self.starts[k] if x > mv]
                if lst:
                    self.starts[k] = min(lst)
                    self.stops[k] = -1
                else:
                    self.starts[k] = -1
                    self.stops[k] = min(self.stops[k])                
        else:
#             self.stops = {k:min(v) for k,v in self.stops.items()}
            self.stops = [min(v) for v in self.stops]
    def __len__(self):
        return self.seqRec.__len__()
#         self = obj
        
# print str(segment(fas[0]).seqRec)
s1 = segment(fas[0])
print len(s1)
# print len(s1.seqRec)
s1 = segment(fas[1])
s2 = segment(fas[2])
print s1.starts
print s1.stops

segs = [segment(fa) for fa in fas]
segs[1] = segment(fas[1],head = 1)
segs[0] = segment(fas[0],head = 1)
#         self.


195
[[114], [-1], [32, 125]]
[-1, -1, -1]


In [124]:
segs = [segment(fa) for fa in fas]
segs[0].stops

{0: 0, 1: 124, 2: 155}

In [101]:
segs[1] = segment(fas[1],head = 1)
print segs[1].starts
print segs[1].stops

{0: -1, 1: 196, 2: 191}
{0: 192, 1: -1, 2: -1}


In [303]:
s = 0
fas = fas0
for fa in fas:
    x = fa.__len__()
    s += x
    print fa.name,x
print "\n",s


FBgn0004666:1 489
FBgn0004666:2 209
FBgn0004666:3 152
FBgn0004666:4 195
FBgn0004666:5 170
FBgn0004666:6 102
FBgn0004666:7 192
FBgn0004666:8 168
FBgn0004666:9 200
FBgn0004666:10 107
FBgn0004666:11 1731

3715


In [163]:
# fas[3].seq.count("CTTTCCCGAT")
fas[3].seq.__str__()

'ATATAGCCAAAACGTGTGCCATGAAGGAGAAAAGCAAAAATGCGGCACGCACGCGACGTGAAAAGGAAAACACGGAATTCTGTGAATTGGCCAAATTACTGCCGCTGCCGGCGGCGATTACTTCGCAACTGGACAAGGCCTCCGTCATCCGGCTGACCACGTCGTATTTGAAAATGCGCCAAGTCTTTCCCGATG'

In [164]:
fas[4].seq.__str__()

'GTCTTGGCGAAGCCTGGGGCTCATCGCCTGCCATGCAACGCGGCGCCACCATCAAGGAGCTGGGCTCCCATCTGCTGCAGACGCTGGACGGATTCATCTTCGTGGTGGCTCCGGATGGCAAAATCATGTACATCTCGGAAACGGCCTCCGTGCATTTGGGCCTCAGTCAG'

In [165]:
fas[5].seq.__str__()

'GTTGAGCTGACGGGCAACTCGATATTCGAGTACATACACAACTACGATCAGGACGAGATGAATGCCATTTTGTCGCTGCATCCGCACATCAACCAGCATCCA'

In [276]:
cds = '''
ATGAAGGAGAA AAGCAAAAAT GCGGCACGCA CGCGACGTGA  550
AAAGGAAAAC ACGGAATTCT GTGAATTGGC CAAATTACTG CCGCTGCCGG  600
CGGCGATTAC TTCGCAACTG GACAAGGCCT CCGTCATCCG GCTGACCACG  650
TCGTATTTGA AAATGCGCCA AGTCTTTCCC GATGGTCTTG GCGAAGCCTG  700
GGGCTCATCG CCTGCCATGC AACGCGGCGC CACCATCAAG GAGCTGGGCT  750
CCCATCTGCT GCAGACGCTG GACGGATTCA TCTTCGTGGT GGCTCCGGAT  800
GGCAAAATCA TGTACATCTC GGAAACGGCC TCCGTGCATT TGGGCCTCAG  850
TCAGGTTGAG CTGACGGGCA ACTCGATATT CGAGTACATA CACAACTACG  900
ATCAGGACGA GATGAATGCC ATTTTGTCGC TGCATCCGCA CATCAACCAG  950
CATCCACTCG CCCAGACGCA CACGCCCATC GGCAGTCCCA ATGGCGTCCA  1000
GCATCCATCC GCCTACGACC ACGATCGCGG ATCGCACACC ATCGAGATCG  1050
AGAAGACCTT CTTCCTGCGC ATGAAGTGCG TCCTGGCCAA AAGGAACGCG  1100
GGCCTCACCA CCTCCGGATT TAAGGTGATA CACTGCTCCG GCTATCTGAA  1150
GGCTCGCATC TATCCCGATC GCGGGGATGG ACAGGGCAGC CTCATCCAGA  1200
ATCTCGGCCT GGTGGCCGTC GGTCACTCGC TGCCTTCATC CGCCATCACG  1250
GAAATCAAGC TGCACCAGAA TATGTTCATG TTCCGGGCCA AGCTGGACAT  1300
GAAGCTCATC TTCTTCGATG CACGCGTATC GCAGCTAACA GGATACGAGC  1350
CGCAGGACCT CATCGAGAAG ACCCTGTATC AGTATATCCA CGCCGCGGAC  1400
ATCATGGCCA TGCGCTGCTC TCATCAAATC CTGCTGTACA AAGGACAAGT  1450
GACCACCAAG TACTACCGCT TCCTCACCAA AGGCGGCGGC TGGGTGTGGG  1500
TGCAGTCGTA CGCGACACTG GTGCACAATT CGCGATCCTC GCGAGAAGTG  1550
TTCATCGTGA GCGTGAACTA TGTGCTGAGC GAACGAGAGG TGAAAGACTT  1600
GGTGCTGAAC GAGATTCAGA CGGGCGTTGT GAAGCGGGAG CCCATATCGC  1650
CGGCGGCTCA AGCGGCACAG GCAGCTCAAG CGGCCCAAGC GGCCCAAGCG  1700
GCACAGGCGG CACAGGCGGC ACATGTGGCT CAAGCGGTGC AGGCCCAGGT  1750
GGTGGTGGTG CCCCAGCAGT CGGTGGTGGT GCAGCCGCAG TGCGCCGGAG  1800
CAACGGGTCA GCCGGTGGGT CCGGGTACGC CAGTTAGTCT GGCCCTCAGT  1850
GCAAGTCCCA AGCTGGATCC GTATTTCGAG CCAGAGCTGC CGCTGCAGCC  1900
GGCCGTAACG CCCGTGCCCC CAACGAATAA TAGTAGTAGT AGTAGCAACA  1950
ATAACAACGG CGTGTGGCAC CACCATCATG TGCAACAGCA GCAGCAGTCC  2000
GGCAGCATGG ATCACGATAG CCTGAGTTAC ACGCAGCTTT ATCCGCCCTT  2050
GAACGACCTG GTGGTGAGCT CGAGCAGCAG TGTGGGCGGT GGCACCGCCT  2100
CAAGTGCGGG CGGTGGATCC AGTGCGTCGG CCTCGTCCTC CGGCGTTTAC  2150
TCCACGGAGA TGCAGTATCC GGACACGACC ACCGGCAACC TGTACTACAA  2200
TAACAACAAT CACTATTACT ATGACTACGA TGCGACGGTG GATGTGGCCA  2250
CCTCGATGAT ACGCCCATTC TCGGCCAATT CGAATAGCTG CTCGAGCAGT  2300
TCGGAGAGCG AAAGGCAGCT GTCCACCGGA AATGCCTCGA TTGTGAACGA  2350
AACATCGCCC TCACAGACGA CCTACAGCGA TCTGAGTCAC AACTTCGAGC  2400
TGAGCTACTT CTCCGACAAC AGTTCGCAAC AGCATCAGCA TCAGCAGCAG  2450
CAGCAGCATC TGATGGAGCA GCAGCATCTG CAATACCAAT ACGCCACCTG  2500
GTGA
'''
re.sub("\d|\n| ",'',cds)
fa = fas[0]
# fa.seq
fa.seq = Bio.Seq.Seq( re.sub("\d|\n| ",'',cds), Bio.Alphabet.DNAAlphabet())
print fa.seq.__len__()

1995


In [304]:
start = 3
fas = fas0[start:]
fas = [fas0[x] for x in [3,4,5,6,8,9,10]]
segs = [segment(fa) for fa in fas]
segs[0] = segment(fas[0],head = 1)
msegs = []
for seg in segs[::-1]:
    print seg.seqRec.name
    print seg.stops
    print seg.shift
#     if len(msegs) == 4:
#         print seg.__len__()
#         for seg2 in msegs:
#             print seg2.stops
#             print find_stop(seg,seg2)
#             print 
#         raise(Exception('hi'))
    seg.stops = furthest_stop( seg, msegs )
    msegs.append(seg)
    print seg.__len__(),seg.shift
#     print len(msegs)
    print seg.stops
    print 
# print seg.stops
print 
print seg.starts 
print seg.stops


FBgn0004666:11
[18, 403, 1070]
0
1731 0
[18, 403, 1070]

FBgn0004666:10
[-1, -1, 11]
2
107 2
[510, 1177, 11]

FBgn0004666:9
[-1, 1, -1]
2
200 2
[1377, 1, 710]

FBgn0004666:7
[-1, 139, 188]
0
192 0
[1569, 139, 188]

FBgn0004666:6
[-1, 7, 2]
0
102 0
[1671, 7, 2]

FBgn0004666:5
[-1, -1, -1]
2
170 2
[1347, 1240, 1841]

FBgn0004666:4
[-1, 58, -1]
0
195 0
[1866, 58, 2036]


[192, -1, 20]
[1866, 58, 2036]


In [318]:
range(9-1,-1,-1)

[8, 7, 6, 5, 4, 3, 2, 1, 0]

In [321]:
url = "http://flybase.org/cgi-bin/getseq.html?source=dmel&id=FBgn0004666&chr=3R&dump=PrecompiledFasta&targetset=exon"
import Bio.SeqIO
import urllib
# with urllib.urlopen(url) as response:
response = urllib.urlopen(url)
fas = [x for x in Bio.SeqIO.parse(response,'fasta')]
f_ind = lambda seq: int( seq.id.split(':')[-1])
fas0 = sorted(fas, key= f_ind)
# fas = fas0
fas = [fas0[x - 1] for x in [4,5,6,8,9,10,11]]
############### test case #################
segs = [segment(fa) for fa in fas]
segs[0] = segment(fas[0],head = 1)
msegs = []
for seg in segs[::-1]:
    print seg.seqRec.name
    print seg.stops
    print seg.shift
#     if len(msegs) == 4:
#         print seg.__len__()
#         for seg2 in msegs:
#             print seg2.stops
#             print find_stop(seg,seg2)
#             print 
#         raise(Exception('hi'))
    seg.stops = furthest_stop( seg, msegs )
    msegs.append(seg)
    print seg.__len__(),seg.shift
#     print len(msegs)
    print seg.stops
    print 
# print seg.stops
print 
print seg.starts 
print seg.stops
act = max([x - y  for x,y in zip(seg.stops,seg.starts)]) + 2 
exp = 1994
assert act == exp,"[FAILED]Expected: %s, Actual:%s"%(exp,act)
# segs.start
# print segs[1].starts`
# print segs[1].stops

FBgn0004666:11
[18, 403, 1070]
0
1731 0
[18, 403, 1070]

FBgn0004666:10
[-1, -1, 11]
2
107 2
[510, 1177, 11]

FBgn0004666:9
[-1, 1, -1]
2
200 2
[1377, 1, 710]

FBgn0004666:8
[-1, 115, 164]
0
168 0
[1545, 115, 164]

FBgn0004666:6
[-1, 7, 2]
0
102 0
[1647, 7, 2]

FBgn0004666:5
[-1, -1, -1]
2
170 2
[1347, 1240, 1817]

FBgn0004666:4
[-1, 58, -1]
0
195 0
[1842, 58, 2012]


[192, -1, 20]
[1842, 58, 2012]


In [333]:
url = "http://flybase.org/cgi-bin/getseq.html?source=dmel&id=FBgn0004666&chr=3R&dump=PrecompiledFasta&targetset=exon"
import Bio.SeqIO
import urllib
# with urllib.urlopen(url) as response:
response = urllib.urlopen(url)
fas = [x for x in Bio.SeqIO.parse(response,'fasta')]
f_ind = lambda seq: int( seq.id.split(':')[-1])
fas0 = sorted(fas, key= f_ind)
# fas = fas0
fas = [fas0[x - 1] for x in [4,5,6,8,9,10,11]]
############### test case #################
segs = [segment(fa) for fa in fas]
heads = [segment(fa,head = 1) for fa in fas]
# segs[0] = segment(fas[0],head = 1)
msegs = []
csegs = []
# for seg in segs[::-1]:
for i in range(len(fas)):
    seg = segs[-1-i]
    hseg = heads[-1-i]
#     print seg.seqRec.name
#     print seg.stops
#     print seg.shift
#     if len(msegs) == 4:
#         print seg.__len__()
#         for seg2 in msegs:
#             print seg2.stops
#             print find_stop(seg,seg2)
#             print 
#         raise(Exception('hi'))
    seg.stops = furthest_stop( seg, msegs )
    hseg.stops = furthest_stop( hseg, msegs )
    msegs.append(seg)
    csegs.append(hseg)
    print seg.__len__(),seg.shift
#     print len(msegs)
    print seg.stops
    print 
# print seg.stops
print 
print hseg.starts 
print hseg.stops
print '\n'.join([str(x.stops) for x in csegs])
act = max([x - y  for x,y in zip(hseg.stops,hseg.starts)]) + 2 
exp = 1994
assert act == exp,"[FAILED]Expected: %s, Actual:%s"%(exp,act)
# segs.start
# print segs[1].starts`
# print segs[1].stops

1731 0
[18, 403, 1070]

107 2
[510, 1177, 11]

200 2
[1377, 1, 710]

168 0
[1545, 115, 164]

102 0
[1647, 7, 2]

170 2
[1347, 1240, 1817]

195 0
[3, 58, 2012]


[192, -1, 20]
[1842, 58, 2012]
[18, 403, 1070]
[510, 1177, 11]
[1377, 1270, 710]
[1545, 115, 164]
[1647, 1279, 2]
[1347, 1240, 1817]
[1842, 58, 2012]


In [330]:
[x.stops for x in csegs]

[[18, 403, 1070],
 [510, 1177, 11],
 [1377, 1270, 710],
 [1545, 115, 164],
 [1737, 139, 188],
 [1839, 1279, 2],
 [1347, 1240, 2009],
 [2034, 58, 2204],
 [18, 2356, 1991],
 [192, 2413, 2048],
 [0, 124, 155]]

In [189]:
pt = " ACGCCACCTG GTGATCCTAA"
pt =   "CAATACCAAT ACGCCACCTG  GTGA"
# pt = "ATGAAGGAGAAAAGCAAAAATGCGGCACGCA"
for fa in fas:
    print fa.seq.count( pt.replace(" ",''))

1
0
0
0
0
0
0
0
0
0
1


In [134]:
seg.starts

{0: -1, 1: 100, 2: 134}

In [116]:
segs = [segment(fa) for fa in fas[start:]]
segs[::-1][0].stops

{0: 18, 1: 403, 2: 1070}

In [225]:
max(1,2,3)

3

In [237]:
# max((1,2,3),(4,1,2))
import numpy as np
x = (1,2,3)
y = (3,1,5)
%timeit np.maximum(x,y) 
%timeit [max(a,b) if b!=-1 else -1 for a,b in zip(x,y)]

The slowest run took 24.55 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.27 µs per loop
The slowest run took 6.11 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.48 µs per loop


In [298]:
def find_stop(seg1,seg2):
    if not seg2:
        return seg1.stops
    stops = seg1.stops[:]
#     print "starting:",seg1.stops
#     print "receiving:",seg2.stops," shift:",seg1.shift
#     d = {}
#     for k,v in seg1.stops.items():
    for k,v in enumerate(seg1.stops):
        if not v == -1:
            continue
        else:
            stops[k] = seg1.__len__() + seg2.stops[(k  - seg1.shift) % 3]
    return stops

def furthest_stop(seg1,segs):
    max_stop = [0, 0, 0]
    if not segs:
        return seg1.stops
#     cand_stops = [find_stop(seg1, seg2).values() for seg2 in segs]
    cand_stops = [find_stop(seg1, seg2) for seg2 in segs]
#     print cand_stops
    for cand_stop in cand_stops:
        max_stop = [max(a,b) if b!=-1 else -1 for a,b in zip(max_stop, cand_stop)]
#         print cand_stop,max_stop
        
#     for seg2 in segs:       
#         stops = find_stop(seg1, seg2)
#         stops_lst += [stops.values()]
#         max()
#     fstops = [max(x) for x in zip(stops_lst)]
#     return dict(zip([0,1,2],max_stop))
    return max_stop

#     for k,v in seg1.keys():
#         seg1

In [248]:
segs[-5].stops
# find_stop

{0: -1, 1: 139, 2: 188}

In [21]:
segs[-1::-1]

In [29]:
segs[0].stops

{0: 0, 1: 124, 2: 155}

In [36]:
fas[-1].reverse_complement()

SeqRecord(seq=Seq('ATTGTTCACCCACGTAATAATTTGTTTATTCAAATTAGAAAAATTCTCGATAAT...GCA', SingleLetterAlphabet()), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])

In [51]:
segs = [segment(fa) for fa in fas]
msegs = [segs[-1]]
for seg in segs[-1::-1]:
    seg.stops = furthest_stop( seg, msegs )
    msegs.append(seg)
    print seg.stops
seg.stops


{0: 18, 1: 403, 2: 1070}
{0: 1177, 1: 125, 2: 11}
{0: 1270, 1: 1, 2: 603}
{0: 186, 1: 115, 2: 164}
{0: 210, 1: 139, 2: 188}
{0: 120, 1: 7, 2: 2}
{0: 1240, 1: 188, 2: 573}
{0: 3, 1: 58, 2: 1265}
{0: 18, 1: 43, 2: 32}
{0: 192, 1: 227, 2: 29}
{0: 0, 1: 124, 2: 155}


{0: 0, 1: 124, 2: 155}

In [56]:
fas

[SeqRecord(seq=Seq('TAGTCACTCATTCGCTCGCCGCCCGCTCGCTGGTTCAGCTGCGCGTCCACGCCG...GAG', SingleLetterAlphabet()), id='FBgn0004666:1', name='FBgn0004666:1', description='FBgn0004666:1 type=exon; loc=3R:13057755..13058243; parent=FBgn0004666,FBtr0082711,FBtr0334614; MD5=11e7e7dfbf81129056e38c394728cf5e; release=r6.17; species=Dmel; length=489;', dbxrefs=[]),
 SeqRecord(seq=Seq('AGTGTTGGATCAAGTCGCACTGCACCCAGTGACCAAGTCGTCGTCGTCGTTGTA...AAA', SingleLetterAlphabet()), id='FBgn0004666:2', name='FBgn0004666:2', description='FBgn0004666:2 type=exon; loc=3R:13062522..13062730; parent=FBgn0004666,FBtr0334613; MD5=3c91cf3f661220346ba19626c7d877fa; release=r6.17; species=Dmel; length=209;', dbxrefs=[]),
 SeqRecord(seq=Seq('GGCTCAGTTGCAAACAGGTGATTGCAGGGATATGAGCAAGTGCTGAGAAGGTGC...ATG', SingleLetterAlphabet()), id='FBgn0004666:3', name='FBgn0004666:3', description='FBgn0004666:3 type=exon; loc=3R:13072399..13072550; parent=FBgn0004666,FBtr0082710; MD5=ab0b8578bd622434d153738549192fac; release=r6.17; specie

In [54]:
fas[0]
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
record = SeqRecord(Seq("MKQHKAMIVALIVICITAVVAALVTRKDLCEVHIRTGQTEVAVF",
                        IUPAC.protein),
                    id="YP_025292.1", name="HokC",
                    description="toxic membrane protein")

In [52]:
segs = [segment(fa.reverse_complement()) for fa in fas]
for seg in segs:
    print seg.stops

{0: 54, 1: 64, 2: 116}
{0: 51, 1: 112, 2: 17}
{0: 45, 1: -1, 2: 17}
{0: 48, 1: 97, 2: -1}
{0: 147, 1: 1, 2: 5}
{0: -1, 1: 52, 2: 11}
{0: 171, 1: 109, 2: 2}
{0: -1, 1: 109, 2: 2}
{0: 99, 1: 16, 2: 86}
{0: 27, 1: 94, 2: 5}
{0: 90, 1: 106, 2: 14}


In [50]:
segs = [segment(fa.reverse_complement()) for fa in fas]
msegs = [segs[-1]]
for seg in segs[-1::-1]:
    seg.stops = furthest_stop( seg, msegs )
    msegs.append(seg)
    print seg.stops
seg.stops


{0: 90, 1: 106, 2: 14}
{0: 27, 1: 94, 2: 5}
{0: 99, 1: 16, 2: 86}
{0: 258, 1: 109, 2: 2}
{0: 171, 1: 109, 2: 2}
{0: 192, 1: 52, 2: 11}
{0: 147, 1: 1, 2: 5}
{0: 48, 1: 97, 2: 209}
{0: 45, 1: 242, 2: 17}
{0: 51, 1: 112, 2: 17}
{0: 54, 1: 64, 2: 116}


{0: 54, 1: 64, 2: 116}

In [13]:
segs = [segment(fa) for fa in fas]
msegs = [segs[-1]]
for seg in segs[:-1:-1]:
    seg.stops = furthest_stop( seg, msegs )
    msegs.append(seg)
    print "seg" in locals()
#     print seg.stops
# seg.stops

In [265]:
seg.stops

{0: 0, 1: 124, 2: 155}

In [251]:
furthest_stop(segs[-5],segs[-3:])

[(0, 191), (1, 139), (2, 188)]

In [1]:
import Bio.SeqIO.FastaIO
# Bio.SeqIO.read(r,'fasta')
help(
    Bio.SeqIO.read
)

Help on function read in module Bio.SeqIO:

read(handle, format, alphabet=None)
    Turns a sequence file into a single SeqRecord.
    
    Arguments:
     - handle   - handle to the file, or the filename as a string
       (note older versions of Biopython only took a handle).
     - format   - string describing the file format.
     - alphabet - optional Alphabet object, useful when the sequence type
       cannot be automatically inferred from the file itself
       (e.g. format="fasta" or "tab")
    
    This function is for use parsing sequence files containing
    exactly one record.  For example, reading a GenBank file:
    
    >>> from Bio import SeqIO
    >>> record = SeqIO.read("GenBank/arab1.gb", "genbank")
    >>> print("ID %s" % record.id)
    ID AC007323.5
    >>> print("Sequence length %i" % len(record))
    Sequence length 86436
    >>> print("Sequence alphabet %s" % record.seq.alphabet)
    Sequence alphabet IUPACAmbiguousDNA()
    
    If the handle contains no recor